`pidgin.rendering` controls the execution of `jinja2` and `mistune`.

In [1]:
    import re, dataclasses, re, functools, IPython, jinja2, inspect, collections, pygments, mistune, inspect, importlib, types, base64, string, traitlets
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]
    ip = IPython.get_ipython()

> The following regular expressions are copied from `mistune`.

`BlockGrammar` includes doctype and prolog parsing in the html and should be used with `BlockLexer`.

In [2]:
    try: from .templating import new_environment
    except:  from templating import new_environment

In [3]:
    class BlockGrammar(mistune.BlockGrammar):
        block_html = re.compile(
            r'^ *(?:%s|%s|%s) *(?:\n{2,}|\s*$)' % (
                r'<!--[\s\S]*?-->|<!DOCTYPE [\s\S]*?>|<\?[\s\S]*?\?>',
                r'<(%s)((?:%s)*?)>([\s\S]*?)<\/\1>' % (mistune._block_tag, mistune._valid_attr),
                r'<%s(?:%s)*?\s*\/?>' % (mistune._block_tag, mistune._valid_attr)))

`InlineGrammar` includes doctype and prolog parsing in the html and should be used with `InlineLexer`.

In [4]:
    class InlineGrammar(mistune.InlineGrammar):
        inline_html = re.compile(
                r'^(?:%s|%s|%s)' % (
                    r'<!--[\s\S]*?-->|<!DOCTYPE [\s\S]*?>|<\?[\s\S]*?\?>',
                    r'<(\w+%s)((?:%s)*?)\s*>([\s\S]*?)<\/\1>' % (
                        mistune._valid_end, mistune._valid_attr),
                    r'<\w+%s(?:%s)*?\s*\/?>' % (mistune._valid_end, mistune._valid_attr)))

In [5]:
    class InlineLexer(mistune.InlineLexer): grammar_class = InlineGrammar
    class BlockLexer(mistune.BlockLexer): grammar_class = BlockGrammar

In [6]:
    class Renderer(mistune.Renderer):
        def text(self, text): return text

In [7]:
    renderer = Renderer(escape=False, parse_block_html=True, parse_inline_html=False)
    markdown = mistune.Markdown(renderer=renderer, block=BlockLexer, inline=InlineLexer)

In [8]:
    @dataclasses.dataclass(order=True)
    class PidginDisplay:
        data: str = ''
        ip: None = None
        ns: None = None
        def __post_init__(self):
            if self.ip and not self.ns: self.ns = self.ip.user_ns
        def _repr_mimebundle_(self, include=None, exclude=None):
            env = getattr(self.ip, 'environment', new_environment(self.ip))
            return {'text/html': markdown.render(
                env.from_string(self.data).render(self.ns)
            ),  'text/markdown': self.data, 'text/plain': self.data}, {}